In [179]:
import json
import requests
import pandas as pd
import numpy as np

In [35]:
with open('..\\config\\token_EmprendimientoColectivo.json') as file:
    data = json.load(file)

In [36]:
class validacion:
    def __init__(self, nombre_api:str, json_malla: bool, nombre_malla : str):
        self.config = "..\\config\\{}.json".format(nombre_api) 
        self.bool_malla = bool
        self.nombre_malla = nombre_malla
        self.token = dict()
        self.dataframe = pd.DataFrame
        self.validacion = pd.DataFrame
        
    def get_token(self):
        try: 
            with open(self.config) as file:
                self.token = json.load(file) 
        except Exception as e:
            print(e)  
    
    def get_dataframe(self):
        self.get_token() 
        
        try: 
            response = requests.get(self.token["url"],
                                    headers = self.token["headers"])
        except Exception as e:
            print("Conexión Fallida")
            print(e)
        
        try:
            self.dataframe = pd.json_normalize(response.json())
        except Exception as e:
            print('No se cargo correctamente como dataframe la información del API')
            print(e)
        

In [37]:
malla = validacion('token_EmprendimientoColectivo', True, 'malla')

In [157]:
ruta_malla = r"C:\Users\andre\OneDrive\Escritorio\DIP\001_MALLA_VALIDACIÓN\formato_validaciones\Malla_Emprendimiento_Colectivo.xlsx"

In [158]:
cond = pd.read_excel(ruta_malla, sheet_name='Validaciones')
val = pd.read_excel(ruta_malla, sheet_name='Valores')
val = val.dropna(subset=['Tipo Validacion'])

In [160]:
def get_list_condiciones(row):
    A = str(row['Condicion'])
    B = row['Tipo Validacion']
    if A =='nan':
        val = None
    else:
        if B == 'int':
            val = [int(i) for i in A.split("|")]
        else:
            val = [str(i) for i in A.split("|")]
    return val

def get_list_valores(row):
    A = str(row['Valores'])
    B = row['Tipo Validacion']
    if A =='nan':
        val = None
    else:
        if B == 'int':
            val = [int(i) for i in A.split("|")]
        elif B == 'str':
            val = [str(i) for i in A.split("|")]
        else:
            val = A
    return val

In [161]:
def create_malla_dict(condiciones, valores):
    malla = dict()
    condiciones['Condicion'] = condiciones.apply(get_list_condiciones, axis = 1)
    valores['Valores'] = valores.apply(get_list_valores, axis = 1)
    try:
        result = condiciones.merge(valores, on = 'Variable', how = 'left')
    except Exception as e:
        print('Problemas con el archivo Excel de Malla de Validación')
        print(e)
        
    for index, row in result.iterrows():
        variable = row['Variable']
        dependiente = row['Dependiente']
        condicion = row['Condicion']
        excluye = row['Excluye Participar']
        obligatoria = row['Obligatorio']
        iand = row['iand']
        valor = row['Valores']
        condicion_valor = row['Tipo Validacion_y']
        
        if variable in malla.keys():
            malla[variable]['condicion'].update({dependiente : condicion})
        else:
            malla[variable] = {
                'condicion': None if pd.isna(dependiente) else {dependiente : condicion},
                'valores': None if pd.isna(condicion_valor) else {'valor':valor, 'Tipo': condicion_valor},
                'iand': False if pd.isna(iand) else True,
                'obligatoria': False if pd.isna(obligatoria) else True,
                'excluida_PTA': False if pd.isna(excluye) else True
                }
    return malla
        

In [162]:
malla = create_malla_dict(cond, val)

In [163]:
malla['ACTIVOS_PROYECTO.Predio o parcela.Estado']

{'condicion': {'ACTIVOS_PROYECTO.Predio o parcela.Seleccione': ['SI']},
 'valores': {'valor': ['Bueno', 'Regular', 'Malo'], 'Tipo': 'str'},
 'iand': False,
 'obligatoria': False,
 'excluida_PTA': True}

In [181]:
def crear_condicion(diccionario, data, iand = False, general = False):
    condicion_total = None
    
    filter1 = data['DESEAPARTICIPAR'] == 'SI'
    filter2 = data['HOGAR_DISPONE_TIERRA'] == True
    filter3 = data['HOGAR_DISPONE_AGUA'] == True
    condicion_general = (filter1) & (filter2) & (filter3)
    
    if diccionario is None:
        if general:
            return condicion_general
        else:
            return None
    else:
        for col, valores in diccionario.items():
            if 'Edad' in col:
                condicion_col = data[col] > valores[0]
            else: 
                condicion_col = data[col].isin(valores)
            if condicion_total is None:
                condicion_total = condicion_col
            else:
                if iand:
                    condicion_total = condicion_total & condicion_col
                else:
                    condicion_total = condicion_total | condicion_col
            if general:
                return condicion_total & condicion_general
            else:
                return condicion_total

In [182]:
def todos_en_valores_permitidos(row, lista, col):
    return all(valor in lista for valor in row[col])

In [183]:
def verificar_valores(diccionario, data, col):
    if diccionario is None:
        return None
    else:
        valores = diccionario['valor']
        tipo = diccionario['Tipo']
        if tipo == 'regex':
            condicion = data[col].astype(str).str.match(valores)
        elif tipo == 'list':
            data[col].apply(todos_en_valores_permitidos, args = (valores, col), axis = 1)
        else:
            condicion = data[col].isin(valores)
    return condicion

In [184]:
def validar_valor(condicion, values, col, data, file):
    file[col] = 0
    if condicion is None:
        val_data = data
    else:
        val_data = data[condicion]
    
    if values is None:
        val_series = val_data[col].isnull().astype(int)
    else:
        val_series = val_data.where(values)[col].isnull().astype(int)
        
    file[col].update(val_series)
    
    return file[col]

In [185]:
def restore_type(data: pd.DataFrame, numeric: list or tuple):
    num_cols = list(set(list(data.select_dtypes(include=np.number).columns))|set(numeric))
    for col in num_cols:
        if col not in ['latitud', 'longitud']:
            try:
                data[col]= data[col].astype('float').astype('Int64')
            except:
                data[col] = np.floor(pd.to_numeric(data[col], errors='coerce')).astype('Int64')
    return data

In [189]:
def malla_validacion(data: pd.DataFrame, guia_validacion : dict) -> pd.DataFrame:
    try:
        numeric = [m for m in [i for i in guia_validacion.keys() if guia_validacion[i]['valores'] is not None] if guia_validacion[m]['valores']['Tipo'] == 'int']
    except Exception as e:
        print('Problemas con la malla de validación entregada')
        print(e)
    data = restore_type(data, numeric)
    columnas = [i for i in data.columns if i in guia_validacion.keys()]
    data = data[columnas]
    store_file = data.copy()
    
    for col in columnas:
        condicion = crear_condicion(guia_validacion[col]['condicion'], data, guia_validacion[col]['iand'], guia_validacion[col]['excluida_PTA'])
        values = verificar_valores(guia_validacion[col]['valores'], data, col)
        store_file[col] = validar_valor(condicion = condicion, values = values, col = col, data = data, file = store_file)
        
    obligatorias = [i for i in guia_validacion.keys() if guia_validacion[i]['obligatoria']== True]
    
    store_file['Documento_Duplicado'] = data['num_documento'].duplicated().astype(int)
    id_hogar = data['id']
    num_doc_representante = data['NUMERODOCUMENTOTITULAR']
    num_doc_integrante = data['num_documento']
    store_file.insert(0, 'ID_HOGAR', id_hogar)
    store_file.insert(1,'NUM_TITULAR', num_doc_representante)
    store_file.insert(2,'NUM_DOC_INTEGRANTE', num_doc_integrante)
    
    obligatorias.append(['Documento Duplicado'])
    
    store_file['Validacion'] = data[obligatorias].sum(axis = 1)
    
    return store_file